In [27]:
from tqdm import tqdm
import pandas as pd
from medvqa.datasets.mimiccxr.preprocessing import (
    extract_report_and_patient_background,
    extract_background_findings_and_impression,
)
from medvqa.datasets.mimiccxr import MIMICCXR_METADATA_CSV_PATH, MIMICCXR_CACHE_DIR, get_reports_txt_paths
from medvqa.utils.files import save_pickle
import os
from pprint import pprint

In [7]:
from importlib import reload
import medvqa

In [66]:
reload(medvqa.datasets.mimiccxr.preprocessing)

<module 'medvqa.datasets.mimiccxr.preprocessing' from '/home/pamessina/medvqa/medvqa/datasets/mimiccxr/preprocessing.py'>

In [3]:
report_paths  = get_reports_txt_paths()

In [35]:
report_paths[0]

'/mnt/data/mimic-cxr/files/p10/p10703179/s58829627.txt'

In [4]:
len(report_paths)

227835

In [5]:
with open(report_paths[-4]) as f:
    print(f.read())

                                 FINAL REPORT
 INDICATION:  ___ year old woman with recurrent endocarditis septic emboli  //
 PNA or abscess
 
 TECHNIQUE:  AP portable chest radiograph
 
 COMPARISON:  ___
 
 FINDINGS: 
 
 No focal consolidation, pleural effusion or pneumothorax identified.  In the
 size the cardiomediastinal silhouette is within normal limits.
 
 Interval removal of the left PICC line.
 
 IMPRESSION: 
 
 No evidence of acute cardiopulmonary disease.



In [169]:
import random
# idx = random.randint(0, len(report_paths)-1)
idx = 135896
print(idx)
extract_background_findings_and_impression(report_paths[idx], True)

135896
                                 FINAL REPORT
 HISTORY:  Incarcerated hernia, sepsis, vent dependence.  Question interval
 change.
 
 CHEST, SINGLE AP PORTABLE VIEW.
 
 COMPARISON:  Chest x-ray from ___ at 4:51 a.m.
 
 Rotated positioning.  A tracheostomy tube is in place.  A right IJ central
 line tip lies in the region of the SVC/RA junction.  The cardiomediastinal
 silhouette is prominent, but unchanged.  There is upper zone re-distribution
 and diffuse vascular blurring, raising the question of CHF/fluid overload. 
 The right costophrenic sulcus remains clear.  There is increased retrocardiac
 density consistent with left lower lobe collapse and/or consolidation,
 unchanged.

OrderedDict([('(HEADERLESS PARAGRAPH)',
              ['                                 FINAL REPORT']),
             ('HISTORY:',
              ['  Incarcerated hernia, sepsis, vent dependence.  Question '
               'interval\n'
               ' change.']),
             ('(AFTER) HISTORY:',
     

{'background': 'history: Incarcerated hernia, sepsis, vent dependence. Question interval change.',
 'findings': 'Rotated positioning. A tracheostomy tube is in place. A right IJ central line tip lies in the region of the SVC/RA junction. The cardiomediastinal silhouette is prominent, but unchanged. There is upper zone re-distribution and diffuse vascular blurring, raising the question of CHF/fluid overload. The right costophrenic sulcus remains clear. There is increased retrocardiac density consistent with left lower lobe collapse and/or consolidation, unchanged.',
 'impression': ''}

In [172]:
!python ../../scripts/mimiccxr/extract_background_findings_and_impression.py

Processing 227835 reports from MIMIC-CXR...
100%|█████████████████████████████████| 227835/227835 [01:33<00:00, 2430.06it/s]
Saved results to /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json


In [176]:
!python ../../scripts/mimiccxr/rank_reports_by_complexity.py \
    --reports-filename "background_findings_and_impression_20230612_174143.json" \
    --chexpert-labels-filename "chexpert_labels_per_report__20220904_113605.pkl" \
    --chest-imagenome-labels-filename "imageId2labels(min_freq=0).pkl" \
    --chest-imagenome-label-names-filename "labels(min_freq=0).pkl" \
    --verbose

Loading reports from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json...
Number of reports: 227835
Loading CheXpert labels from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/chexpert_labels_per_report__20220904_113605.pkl...
CheXpert labels shape: (227835, 14)
Loading Chest ImaGenome labels from imageId2labels(min_freq=0).pkl...
Chest ImaGenome labels shape: (227835, 736)
Concatenating the labels...
Labels shape: (227835, 750)
Counting label occurrences...
Finding bins for reports in parallel with 10 workers...
len(global_label_indices): 93
Sorting each bin by complexity...
Mean CheXpert label sum: 2.485403910724867
Mean Chest ImaGenome label sum: 19.943195733754692
Mean report length: 414.6148572431804
Bin sizes and label names:
0: 2 ('texture', 'calcified')
1: 2 ('texture', 'interstitial')
2: 3 ('texture', 'alveolar')
3: 11 ('texture', 'opacity')
4: 56 ('anatomicalfinding', 'diaphragmatic eventrati

In [182]:
!python ../../scripts/mimiccxr/rank_patient_backgrounds_by_complexity.py \
    --reports-filename "background_findings_and_impression_20230612_174143.json"

Loading reports from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json...
Number of reports: 227835
100%|███████████████████████████████| 227835/227835 [00:01<00:00, 142881.19it/s]
Vocabulary size: 30978
Sorting backgrounds by the inverse of the frequency of their words...
Mean word frequency: 113.85576860998128
Saved ranked background indices to /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/ranked_background_idxs_(background_findings_and_impression_20230612_174143.json).pkl


In [183]:
from medvqa.utils.files import load_pickle
# ranked_indices = load_pickle('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/ranked_report_indices_1.0_1.0_1.0.pkl')
ranked_indices = load_pickle('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/ranked_background_idxs_(background_findings_and_impression_20230612_174143.json).pkl')

In [1]:
from medvqa.utils.files import load_json
tmp = load_json('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json')

In [1]:
from medvqa.utils.files import load_jsonl
tmp = load_jsonl('/home/pamessina/medvqa-workspace/tmp/mimiccxr/openai/gpt-3.5-turbo_parsed_reports__backup.jsonl')

In [2]:
len(tmp)

227835

In [4]:
tmp[0]

{'background': 'indication: New dyspnea on exertion.',
 'findings': 'Heart size is normal. There are mild aortic knob calcifications. Cardiomediastinal silhouette and hilar contours are otherwise unremarkable. Lungs are clear. Pleural surfaces are clear without effusion or pneumothorax.',
 'impression': 'No acute cardiopulmonary abnormality.',
 'path': '/mnt/data/mimic-cxr/files/p10/p10703179/s58829627.txt'}

In [8]:
from tqdm import tqdm

tmp2 = [None] * len(tmp)
for i, x in tqdm(enumerate(tmp), mininterval=2):
    tmp2[i] = {**x}
    with open(x['path']) as f:
        tmp2[i]['original_report'] = f.read()

227835it [12:16, 309.42it/s]


In [9]:
from medvqa.utils.files import save_json

In [10]:
save_json(tmp2, '/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/reports_backgrounds_findings_impressions.json')

In [14]:
!ls -lh /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/reports_backgrounds_findings_impressions.json

-rw-rw-r-- 1 pamessina pamessina 288M Aug 14 16:15 /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/reports_backgrounds_findings_impressions.json


In [11]:
tmp3 = load_json('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/reports_backgrounds_findings_impressions.json')

In [13]:
tmp3[-1]

{'background': 'indication: Chest pain.',
 'findings': 'The cardiac silhouette size is top normal. The aorta is mildly unfolded. Hilar contours are normal. The pulmonary vascularity is normal and the lungs are clear. No pleural effusion or pneumothorax is seen. There are mild degenerative changes of the thoracic spine.',
 'impression': 'No acute cardiopulmonary abnormality.',
 'path': '/mnt/data/mimic-cxr/files/p19/p19216802/s51441827.txt',
 'original_report': '                                 FINAL REPORT\n INDICATION:  Chest pain.\n \n COMPARISON:  None.\n \n PA AND LATERAL VIEWS OF THE CHEST:  The cardiac silhouette size is top normal.\n The aorta is mildly unfolded. Hilar contours are normal. The pulmonary\n vascularity is normal and the lungs are clear.  No pleural effusion or\n pneumothorax is seen.  There are mild degenerative changes of the thoracic\n spine.  \n \n IMPRESSION:  No acute cardiopulmonary abnormality.\n'}

In [51]:
re.match(r"^.+\s#(pos|neg)$", "   #neg")

<re.Match object; span=(0, 7), match='   #neg'>

In [53]:
tmp[68626]

{'background': 'indication: y/o M with CAD, ischemic cardiomyopathy (EF % TTE ), HTN, HLD, moderate AS who presented with worsening fatigue, abdominal distention, leg swelling, decreased exercise tolerance, and decreased appetite, likely secondary to decompensated congestive heart failure. // Interval change? Interval change?.',
 'findings': 'Tip of the new intra-aortic balloon pump is less than a cm from the apex of the aortic knob, proximally 2 cm above optimal placement. Pulmonary edema is mild, substantially improved since . Lung volumes are still very low, with particular elevation of the right hemidiaphragm. Cardiomegaly is moderate to severe. Left internal jugular Swan-Ganz catheter ends in the main pulmonary artery. A dual channel right jugular line ends in the low SVC. Right pleural drainage catheter unchanged in position. No pneumothorax or appreciable pleural effusion on either side. Stomach is moderately distended with air and fluid. No nasogastric tube is seen.',
 'impress

In [41]:
import re
import json
match = re.search(r"\[\s*(\".+?\",?\s*)*\]", """
[] (no facts can be extracted without specific information)""")
json.loads(match.group(0))

[]

In [27]:
eval(match.group(0))

['female',
 'fall',
 'headstrike',
 'no loss of consciousness',
 'rule out intracranial hemorrhage',
 'rule out pneunomia']

In [17]:
"""[
"female",
"fall",
"headstrike",
"no loss of consciousness",
"rule out intracranial hemorrhage",
"rule out pneunomia"
]"""

'[\n"female",\n"fall",\n"headstrike",\n"no loss of consciousness",\n"rule out intracranial hemorrhage",\n"rule out pneunomia"\n]'

In [14]:
# tmp[ranked_indices[3011]]
tmp[106263]

{'background': 'indication: <Clinical Indication><Reason For Exam>.',
 'findings': '<Procedure Description>. Severe subcutaneous emphysema in the right chest and both sides of the neck has probably not worsened since after increasing substantially relative to . Small right hydro pneumothorax collected laterally is unchanged, pleural drain still in the right chest. Pulmonary vascular congestion has increased since . Heart size is normal. Epidural infusion catheter is coiled in the midline.',
 'impression': '',
 'path': '/mnt/data/mimic-cxr/files/p14/p14990450/s55851998.txt'}

In [180]:
from nltk.tokenize import wordpunct_tokenize
wordpunct_tokenize('indication: year old woman s/p chest tube // chest tube placement.')

['indication',
 ':',
 'year',
 'old',
 'woman',
 's',
 '/',
 'p',
 'chest',
 'tube',
 '//',
 'chest',
 'tube',
 'placement',
 '.']

In [178]:
idx = ranked_indices[82]
print(idx)
extract_background_findings_and_impression(report_paths[idx], True)

141869
                                 FINAL REPORT
 EXAMINATION:  CHEST (PORTABLE AP)
 
 INDICATION:  ___ year old woman s/p chest tube  // chest tube placement
 
 COMPARISON:  Chest x-ray from ___ at 18:18
 
 FINDINGS: 
 
 Rotated positioning.
 
 A Port-A-Cath type catheter is present, tip probably also overlies the distal
 SVC.
 
 Again seen is cardiomegaly, with a globular configuration.
 
 Also again seen is left lower lobe collapse and/or consolidation with air
 bronchograms obscuration of left hemidiaphragm.  The possibility of associated
 left pleural fluid cannot be excluded.  The upper portion left lung is grossly
 clear without CHF or focal infiltrate.  No left-sided pneumothorax.
 
 On the right, 2 chest tubes are seen at the base of the lung.  Fluid and air
 may be present tracking at the right lung apex.
 
 Compare to the prior study, there is a new large lobulated opacity along the
 right mediastinum and in the right mid zone.  A small amount pleural fluid is
 seen in t

{'background': 'indication: year old woman s/p chest tube // chest tube placement.',
 'findings': 'Rotated positioning. A Port-A-Cath type catheter is present, tip probably also overlies the distal SVC. Again seen is cardiomegaly, with a globular configuration. Also again seen is left lower lobe collapse and/or consolidation with air bronchograms obscuration of left hemidiaphragm. The possibility of associated left pleural fluid cannot be excluded. The upper portion left lung is grossly clear without CHF or focal infiltrate. No left-sided pneumothorax. On the right, 2 chest tubes are seen at the base of the lung. Fluid and air may be present tracking at the right lung apex. Compare to the prior study, there is a new large lobulated opacity along the right mediastinum and in the right mid zone. A small amount pleural fluid is seen in the right sulcus. The etiology of this is unclear. Most likely etiology is some loculated fluid along the medial surface of the right lung, together with f

In [1103]:
_re_paragraph_breaks.split('''WET READ: ___ ___ ___ 6:17 PM
  Comp:  ___
  
  Interval placement of right pigtail catheter
  Persistent right costophrenic angle opacity and more medial right basilar
  opacity as well.
  Thin lucency along the lateral chest wall may represent air introduced during
  the procedure.  No large pneumothorax is present.
 ______________________________________________________________________________
                                 FINAL REPORT
 EXAMINATION:  CHEST (PORTABLE AP)
 
 INDICATION:  ___ year old man with chest tube  // ? ptx      ? ptx
 
 COMPARISON:  Chest radiographs most recently ___.
 
 IMPRESSION: 
 
 Since ___:55, with insertion of a right basal pigtail pleural
 drainage catheter, most of the previous moderate right pleural effusion has
 been evacuated.  Small localized pneumothorax is new.  Opacification in the
 right lower lobe could be atelectasis alone or there could be pneumonia. 
 Previous mild pulmonary edema has improved.  Left lower lobe is chronically
 atelectatic reflected in elevation of the left hemidiaphragm and leftward
 mediastinal shift.  Overall lung volumes are large, due to emphysema or
 obstructive lung disease.  Moderate cardiomegaly is underestimated as a
 result.''')

['WET READ: ___ ___ ___ 6:17 PM\n  Comp:  ___',
 '\n  \n  ',
 '  \n  ',
 None,
 'Interval placement of right pigtail catheter\n  Persistent right costophrenic angle opacity and more medial right basilar\n  opacity as well.\n  Thin lucency along the lateral chest wall may represent air introduced during\n  the procedure.  No large pneumothorax is present.',
 None,
 None,
 '\n ______________________________________________________________________________\n',
 '                                 FINAL REPORT\n EXAMINATION:  CHEST (PORTABLE AP)',
 '\n \n ',
 ' \n ',
 None,
 'INDICATION:  ___ year old man with chest tube  // ? ptx      ? ptx',
 '\n \n ',
 ' \n ',
 None,
 'COMPARISON:  Chest radiographs most recently ___.',
 '\n \n ',
 ' \n ',
 None,
 'IMPRESSION:',
 ' \n \n ',
 ' \n ',
 None,
 'Since ___:55, with insertion of a right basal pigtail pleural\n drainage catheter, most of the previous moderate right pleural effusion has\n been evacuated.  Small localized pneumothorax is new.  Opac

In [1015]:
_p = "  SA AAaa:\n RES-  - R- SE: asdfasdf sdfds  \n ASDKFD: /ASD FASDF: \nBaaa:"
[_p[i.span()[0]:i.span()[1]] for i in _re_header.finditer(_p)]

['  SA AAaa:', '\n RES-  - R- SE:', '\n ASDKFD:', '\nBaaa:']

In [1969]:
from collections import OrderedDict
import re
_re_header = re.compile(r'(^|\n)\s*([A-Z][a-zA-Z]*(( |-|&)+[a-zA-Z]+)*?:)')
_re_paragraph_breaks = re.compile(r'(\s*\n(\s*\n\s*)+)|(\n\s*_+\s*\n)')

def split_report_into_sections(report_path, debug=False):
    with open(report_path) as f:
        text = f.read()
    if debug:
        print(text)
    paragraphs = _re_paragraph_breaks.split(text)
    sections = OrderedDict()
    last_k = None
    for p in paragraphs:
        if p is None: continue
#         if debug:
#             print('------------------')
#             print(p)
        spans = [i.span() for i in _re_header.finditer(p)]
        if len(spans) > 0:            
            if spans[0][0] > 0:
                if last_k is None:
                    k = '(HEADERLESS PARAGRAPH)'
                else:
                    k = f'(AFTER) {last_k}'
                try:
                    tmp = sections[k]
                except KeyError:
                    tmp = sections[k] = []
                tmp.append(p[:spans[0][0]])
            for i, span in enumerate(spans):
                k = p[span[0]:span[1]]
                k = ' '.join(k.split()).strip()
                v = p[span[1]:] if i+1 == len(spans) else p[span[1]:spans[i+1][0]]
                # sections[k] = v
                try:
                    tmp = sections[k]
                except KeyError:
                    tmp = sections[k] = []
                tmp.append(v)
                last_k = k
        else:
            if last_k is None:
                k = '(HEADERLESS PARAGRAPH)'
            else:
                k = f'(AFTER) {last_k}'
            try:
                tmp = sections[k]
            except KeyError:
                tmp = sections[k] = []
            tmp.append(p)
    return sections

In [364]:
# split_report_into_sections(rp, True)

In [2019]:
SECTION_HEADERS_FOR_REPORT = set([
    'FINDINGS:',
    'FINDNINGS:',
    'FINGDINGS:',
    'FINIDNGS:',
    'Findings:',
    'FINDINS:',
    'FINDING:',
    'FINDINDGS:',
    'FIMPRESSION:',
    'FINSINGS:',
    'FINDIGNS:',
    'FINDINGS and IMPRESSION:',
    'MPRESSION:',
    'IMPRESSION:',
    'IMPRESSON:',
    'IMPRESSIONS:',
    'IMPRESION:',
    'IMPESSION:',
    'IMPRSSION:',
    'IMPRESSOIN:',
    'Findings and Impression:',
    'FINDINGS AND IMPRESSION:',
    'Impression:',
#     'EXAMINATION:',
    'REPORT:',
    'CONCLUSION:',
    'Conclusion:',
    'THEY REPORT TEXT FOLLOWS:',
    'PORTABLE SUPINE FRONTAL VIEW OF THE CHEST:',
    'FRONTAL AND LATERAL CHEST RADIOGRAPHS:',
    'FRONTAL AND LATERAL CHEST RADIOGRAPH:',
    'FRONTAL AND LATERAL VIEWS OF THE CHEST:',
    'PA AND LATERAL CHEST RADIOGRAPH:',
    'PA AND LATERAL VIEWS OF THE CHEST:',
    'PA AND LATERAL:',
    'AP AND LATERAL:',
    'PA AND LATERAL CHEST:',
    'FRONTAL AND LATERAL CHEST:',
    'TWO VIEWS OF THE CHEST:',
    'PORTABLE AP CHEST:',
    'PORTABLE AP CHEST RADIOGRAPH:',
    'PA AND LATERAL CHEST RADIOGRAPHS:',
    'UPRIGHT PORTABLE RADIOGRAPH OF THE CHEST:',
    'PORTABLE RADIOGRAPH OF THE CHEST:',
    'FRONTAL CHEST RADIOGRAPH:',
    'FRONTAL UPRIGHT PORTABLE CHEST:',
    'PORTABLE FRONTAL CHEST RADIOGRAPH:',
    'PORTABLE UPRIGHT FRONTAL VIEW OF THE CHEST:',
    'UPRIGHT AP VIEW OF THE CHEST:',
    'PORTABLE CHEST RADIOGRAPH:',
    'SINGLE AP PORTABLE VIEW:',
    'SINGLE PORTABLE CHEST RADIOGRAPH:',
    'SINGLE PORTABLE VIEW OF THE CHEST:',
    'UPRIGHT FRONTAL CHEST RADIOGRAPHS:',
    'AP:',
    'AP UPRIGHT:',
    'AP CHEST:',
    'CHEST RADIOGRAPH:',
    'TWO VIEWS OF THE THORACIC SPINE:',
    'TWO VIEWS:',
    'AP AND LATERAL VIEWS OF THE CHEST:',
    'ERECT FRONTAL CHEST RADIOGRAPH:',
    'PORTABLE FRONTAL VIEW OF THE CHEST:',
    'PORTABLE RADIOGRAPH:',
    'CHEST AP:',
    'ONE VIEW OF THE CHEST:',
    'AP FILM:',
    'SUPINE AP VIEW OF THE CHEST:',
    'AP VIEW OF THE CHEST:',
    'AP AND LATERAL CHEST RADIOGRAPHS:',
    'AP PORTABLE CHEST:',
    'PORTABLE AP UPRIGHT CHEST RADIOGRAPH:',
    'SINGLE FRONTAL VIEW OF THE CHEST:',
    'DOUBLE CHEST RADIOGRAPH:',
    'CHEST TWO VIEWS:',
    'SUPINE PORTABLE CHEST RADIOGRAPH:',
    'SINGLE PORTABLE AP CHEST RADIOGRAPH:',
    'PORTABLE AP VIEW OF THE CHEST:',
    'PORTABLE ERECT RADIOGRAPH:',
    'PFI:',
    'PORTABLE SUPINE RADIOGRAPH OF THE CHEST:',
    'SINGLE AP VIEW:',
    'PORTABLE UPRIGHT AP VIEW OF THE CHEST:',
    'FRONTAL VIEW OF THE CHEST:',
    'SUPINE PORTABLE CHEST:',
    'PORTABLE UPRIGHT RADIOGRAPH OF THE CHEST:',
    'SINGLE AP VIEW OF THE CHEST:',
    'SINGLE AP PORTABLE CHEST RADIOGRAPH:',
    'PORTABLE AP AND LATERAL CHEST RADIOGRAPH:',
    'CHEST PORTABLE:',
    'PA AND LATERAL VIEWS:',
    'AP SUPINE:',
    'CHEST SINGLE VIEW:',
    'FOUR IMAGES:',
    'PORTABLE SUPINE CHEST RADIOGRAPH:',
    'SEMIUPRIGHT PORTABLE RADIOGRAPH OF THE CHEST:',
    'SINGLE FRONTAL CHEST RADIOGRAPHS:',
    'FRONTAL SUPINE PORTABLE CHEST:',
    'CHEST PA AND LAT RADIOGRAPH:',
    'AP UPRIGHT AND LATERAL:',
    'AP PORTABLE FRONTAL CHEST RADIOGRAPH:',
    'SINGLE PORTABLE FRONTAL VIEW OF THE CHEST:',
#     'RIGHT RIBS:',
    'VIEWS:',
    'FRONTAL AND LATERAL VIEWS THE CHEST:',
    'AP FRONTAL CHEST RADIOGRAPH:',
    'KYPHOTIC POSITIONING:',
    'PORTABLE UPRIGHT CHEST RADIOGRAPH:',
    'UPRIGHT FRONTAL AND LATERAL CHEST RADIOGRAPHS:',
    'SUPINE CHEST RADIOGRAPH:',
    'AP PORTABLE:',
    'OSSEOUS STRUCTURES:',
    'SINGLE VIEW:',
    'SINGLE PORTABLE UPRIGHT VIEW OF THE CHEST:',    
    'SINGLE FRONTAL PORTABLE VIEW OF THE CHEST:',
    'FRONTAL AND LATERAL RADIOGRAPHS:',
    'RADIOGRAPH:',
    'SINGLE AP SUPINE PORTABLE VIEW:',
    'LORDOTIC POSITIONING:',
    'UPRIGHT AP AND LATERAL VIEWS OF THE CHEST:',
    'SUPINE PORTABLE FRONTAL CHEST RADIOGRAPH:',
    'AP CHEST RADIOGRAPH:',
    'FRONTAL AND LATERAL VIEWS CHEST:',
    'FRONTAL PORTABLE CHEST:',
    'FRONTAL PORTABLE SUPINE CHEST:',
    'UPRIGHT FRONTAL VIEW OF THE CHEST:',
    'PORTABLE SUPINE AP CHEST RADIOGRAPH:',
    'PA AND LATERAL VIEWS OF CHEST:',
    'TECHNIQUE PA AND LATERAL VIEWS OF THE CHEST:',
    'AP FRONTAL AND LATERAL CHEST RADIOGRAPHS:',
    'CHEST PA:',
    'AP PORTABLE CHEST RADIOGRAPH:',
    'PORTABLE AP FRONTAL CHEST RADIOGRAPH:',
    'SINGLE AP PORTABLE VIEW WHICH INCLUDES THE UPPER ABDOMEN:',
    'LATERAL VIEWS OF THE CHEST:',
    'SUPINE FRONTAL VIEW OF THE CHEST:',
    'FRONTAL CHEST RADIOGRAPH WITH THE PATIENT IN SUPINE AND UPRIGHT POSITIONS:',
    'AP UPRIGHT VIEW OF THE CHEST:',
    'AP AND LATERAL CHEST RADIOGRAPH:',
    'PORTABLE SEMI-ERECT FRONTAL CHEST RADIOGRAPH:',
    'PORTABLE SUPINE FRONTAL CHEST RADIOGRAPH:',
    'FRONTAL SEMI-UPRIGHT PORTABLE CHEST:',
    'SEMI-ERECT PORTABLE AP CHEST RADIOGRAPH:',
    'PORTABLE SEMI-UPRIGHT AP CHEST RADIOGRAPH:',
    'PA AND LATERAL RADIOGRAPHS OF THE CHEST:',
    'FRONTAL PORTABLE UPRIGHT RADIOGRAPH:',
    'UPRIGHT PORTABLE CHEST RADIOGRAPH:',
    'AP PORTABLE UPRIGHT CHEST RADIOGRAPH:',
    'PORTABLE UPRIGHT RADIOGRAPH CHEST:',
    'SINGLE AP PORTABLE VIEW OF THE CHEST:',
    'SEMI-ERECT FRONTAL CHEST RADIOGRAPH:',
    'SEMI-UPRIGHT AP VIEW OF THE CHEST:',
    'FRONTAL SEMI-SUPINE PORTABLE CHEST:',
    'FRONTAL PORTABLE UPRIGHT CHEST:',
    'CHEST AP SUPINE:',
    'AP UPRIGHT AND LATERAL CHEST RADIOGRAPHS:',
    'PORTABLE UPRIGHT AP CHEST RADIOGRAPH:',
    'PORTABLE UPRIGHT AP VIEW OF THE ABDOMEN:',
    'PORTABLE AP SEMI-UPRIGHT CHEST RADIOGRAPH:',
    'PA AND LAT CHEST RADIOGRAPH:',
    'PORTABLE AP SEMI-ERECT RADIOGRAPH:',
    'Chest:',
    'PORTABLE SEMI-UPRIGHT FRONTAL CHEST RADIOGRAPH:',
    'PA AND LAT:',
    'FRONTAL AP AND LATERAL CHEST:',
    'SEMI-ERECT PORTABLE CHEST RADIOGRAPH:',
#     'Support Devices:',
#     'Lungs:', 
#     'Pleura:', 
#     'Heart:',
#     'Lines and Tubes:',
#     'Lines and tubes:',
#     'Mediastinum:',
#     'Mediastinum and hila:',
#     'Osseous structures:', 
#     'Other findings:',
#     'Bony thorax:',
#     'Pelvis:',
#     'Abdomen:',
    'AP UPRIGHT CHEST RADIOGRAPH:',
    'PA AND LATERAL UPRIGHT CHEST RADIOGRAPHS:',
    'TWO PORTABLE ERECT VIEWS OF THE CHEST:',
    'CHEST AND UPPER ABDOMEN:',
    'Portable chest:',
    'PA and lateral chest reviewed in the absence of prior chest imaging:',
    'CHEST AND PELVIS FILMS:',
    'Chest radiographs:',
    'FRONTAL and LATERAL VIEWS OF THE CHEST:',
    'PORTABLES SEMI-ERECT CHEST RADIOGRAPH:',
    'BEDSIDE FRONTAL CHEST RADIOGRAPH:',
    'SEMI-ERECT PORTABLE CHEST:',
    'SEMI-UPRIGHT PORTABLE RADIOGRAPH OF THE CHEST:',
    'AP UPRIGHT VIEWS OF THE CHEST DURING INSPIRATION AND EXPIRATION:',
    'SEMI-ERECT PORTABLE FRONTAL CHEST RADIOGRAPH:',
    'PORTABLE SEMI-UPRIGHT VIEW OF THE CHEST:',
    'FRONTAL SUPINE PORTABLE VIEW OF THE CHEST:',
    'PA & LATERAL VIEW OF THE CHEST:',
    'PA and lateral views of the chest:',
    'In comparison to previous radiographs:',
    'SUPINE CHEST:',
    'TWO PORTABLE AP VIEWS OF THE CHEST:',
    'PORTABLE SUPINE CHEST:',
    'SUPINE PORTABLE FRONTAL VIEW OF THE CHEST:',
    'BEDSIDE AP UPRIGHT RADIOGRAPH OF THE CHEST:',
    'Again demonstrated are:',
    'RIGHT AND LEFT FRONTAL OBLIQUE VIEWS OF THE CHEST:',
    'PORTABLE SEMI-UPRIGHT RADIOGRAPH OF THE CHEST:',
    'AP UPRIGHT AND LATERAL VIEWS OF THE CHEST:',
    'AP SUPINE CHEST RADIOGRAPH:',
    'FRONTAL VIEWS OF THE CHEST:',
    'PA AND LATERAL CHEST FILMS:',
    'BEDSIDE AP RADIOGRAPH OF THE CHEST:',
    'PORTABLE AP UPRIGHT RADIOGRAPH OF THE CHEST:',
    'CHEST RADIOGRAPHS:',
    'PA AND LATERAL VIEWS CHEST:',
    'PORTABLE SEMI-ERECT AP CHEST RADIOGRAPH:',
    'SINGLE FRONTAL CHEST RADIOGRAPH:',
    'PORTABLE AP FRONTAL VIEW OF THE CHEST:',
    'SUPINE PORTABLE CHEST RADIOGRAPHS:',
    'PORTABLE AP UPRIGHT VIEW OF THE CHEST:',
    'PORTABLE PA CHEST RADIOGRAPH:',
    'AP view of chest:',
    'UPRIGHT AP AND LATERAL VIEWS OF CHEST:',
    'PORTABLE SEMI-ERECT CHEST RADIOGRAPH:',
    'SEMI-UPRIGHT PORTABLE CHEST RADIOGRAPH:',
    'SINGLE AP ERECT PORTABLE VIEW OF THE CHEST:',
    'SINGLE UPRIGHT PORTABLE CHEST RADIOGRAPH:',
    'AP and lateral views of the chest:',
    'UPRIGHT AP AND LATERAL CHEST RADIOGRAPH:',
    'UPRIGHT FRONTAL AND LATERAL VIEWS OF THE CHEST:',
    'PORTABLE UPRIGHT FRONTAL CHEST RADIOGRAPH:',
    'FRONTAL AND LATERAL VIEWS OF CHEST:',
    'FRONTAL CHEST RADIOGRAPHS:',
    'PA and lateral views of chest:',
    'SINGLE PORTABLE UPRIGHT CHEST RADIOGRAPH:',
    'SUPINE AP PORTABLE CHEST RADIOGRAPH:',
    'Additional findings:',
    'SUPINE PORTABLE AP CHEST RADIOGRAPH:',
    'SINGLE SEMI-ERECT PORTABLE VIEW OF THE CHEST:',
    'PORTABLE SEMI-UPRIGHT CHEST RADIOGRAPH:',
    'FRONTAL LATERAL CHEST RADIOGRAPH:',
    'SEMI-ERECT PORTABLE VIEW OF THE CHEST:',
    'PA AND LATERAL FILMS OF THE CHEST:',
    'SEMI-UPRIGHT AP AND LATERAL VIEWS OF THE CHEST:',
    'CHEST PA AND LATERAL RADIOGRAPH:',
    'FRONTAL AND LATERAL UPRIGHT CHEST RADIOGRAPH:',
    'PORTABLE SEMI-ERECT FRONTAL CHEST RADIOGRAPHS:',
    'OPINION:',
    'FRONTAL PORTABLE CHEST RADIOGRAPH:',
    'SEMIERECT PORTABLE RADIOGRAPH OF THE CHEST:',
    'PORTABLE SEMI-ERECT AP AND PA CHEST RADIOGRAPH:',
    'FRONTAL SEMI UPRIGHT PORTABLE CHEST:',
    'SEMIERECT AP VIEW OF THE CHEST:',
    'AP VIEW AND LATERAL VIEW OF THE CHEST:',
    'PORTABLE SEMI-ERECT CHEST:',
    'PORTABLE SEMI-UPRIGHT FRONTAL VIEW OF THE CHEST:',
    'AP SEMI-ERECT CHEST RADIOGRAPH:',
    'SINGLE AP UPRIGHT PORTABLE CHEST RADIOGRAPH:',
    'SINGLE SUPINE PORTABLE VIEW OF THE CHEST:',
    'PA AND AP CHEST RADIOGRAPH:',
    'SINGLE PORTABLE AP VIEW OF THE CHEST:',
    'PORTABLE AP CHEST RADIOGRAPHS:',
    'PORTABLE SEMI-UPRIGHT FRONTAL CHEST RADIOGRAPHS:',
    'SEMI-UPRIGHT AP AND LATERAL CHEST RADIOGRAPHS:',
    'SINGLE PORTABLE CHEST X-RAY:',
    'PORTABLE SEMI-UPRIGHT RADIOGRAPH:',
    'AP portable view of the chest:',
    'AP chest reviewed in the absence of prior chest radiographs:',
    'PA and lateral chest reviewed in the absence of prior chest radiographs:',
])
SECTION_HEADERS_FOR_REPORT.update([
    f'(AFTER) {key}' for key in SECTION_HEADERS_FOR_REPORT
])    
SECTION_HEADERS_FOR_REPORT.update([
    '(AFTER) WET READ:',
    '(AFTER) COMPARISON:',
    '(AFTER) COMPARISON FILM:',
])

SECTION_HEADERS_FOR_REPORT_BACKUP = set([
    '(AFTER) REASON FOR EXAMINATION:',
    '(AFTER) REASON FOR EXAM:',
    '(AFTER) REASON FOR INDICATION:',
    '(AFTER) INDICATION:',
    '(AFTER) CLINICAL HISTORY:',
    '(AFTER) HISTORY:',
    '(AFTER) TECHNIQUE:',
    '(AFTER) TYPE OF EXAMINATION:',
    '(AFTER) EXAM:',
    '(HEADERLESS PARAGRAPH)',
])

SECTION_HEADERS_FOR_PATIENT_BACKGROUND = set([
    'REASON FOR EXAMINATION:',
    'REASON FOR EXAM:',
    'REASON FORE EXAM:',
    'REASON FOR THE EXAM:',
    'Reason for exam:',
    'REASON FOR INDICATION:',
    'INDICATION:',
    'INDCATION:',
    'IDICATION:',
    'NDICATION:',
    'INDICATIONS:',
    'Indication:',
    'ADDENDUM Indication:',
#     'Comparisons:',
#     'COMPARISON:',
#     'COMPARISONS:',
#     'Comparison:',
#     'COMPARISION:',
#     'COMPARISON STUDY:',
#     'COMPARISON TO PRIOR STUDY:',
#     'COMPARISON STUDIES:',
#     'COMPARISON EXAMS:',
#     'COMPARRISON:',
#     'COMPARISON CHEST:',
#     'Comp:',
#     'PRIOR EXAM:',
    'WET READ:',
    '(AFTER) WET READ:',
    'RESIDENT WET READ:',
    'PRELIMINARY RESIDENT WET READ:',
    'PRELIMINARY REPORT:',
    'CLINICAL HISTORY:',
    'CLINICAL HISTORY History:',
    'CLINICAL INDICATION:',
    'Clincal indication:',
    'CLINICAL INFORMATION:',
    'CLINIC INDICATION:',
    'ADDITIONAL CLINICAL HISTORY PROVIDED:',
    'HISTORY:',
    'History:',
    'ADDENDUM INDICATION:',
    'PATIENT HISTORY:',
    'COR:',
    'CHOCTAW:',
    'ICHCt abd:',
    'NCHCT:',
    'ILLNESS:',
    'Surg:',
    'Pre-op:',
    'Pre-op CXR Surg:',
    'Pre-op Surg:',
    'Pre-op chest xray Surg:',
    'Pre-operative planning Surg:',
    'Please obtain pre-op CXR Surg:',
    'CXR Surg:',
    'BKA Surg:',
    'X-ray Surg:',
    'Story:',
    'Head CT:',
    'CT Head:',
    'Please eval for acute process Surg:',
    'Please assess for any abnormalities Surg:',
    'Please evaluate Surg:',
])

IGNORABLE_HEADERS = set([
     'EXAM:', 'STUDY:', 'TYPE OF EXAMINATION:',
     'DATE:', 'RECOMMENDATIONS:', 'CXR:', 'TYPE OF THE EXAMINATION:',
     'RECOMMENDATION:', 'CT:', 'CTA:', 'RUQ:', 'QUESTIONS TO BE ANSWERED:',
    'CLINICAL INFORMATION & QUESTIONS TO BE ANSWERED:',
    "CC:", 'Compressing:', 'Contact name:', 'Common:',
])

SECTION_HEADERS_FOR_PATIENT_BACKGROUND.update([x.lower() for x in SECTION_HEADERS_FOR_PATIENT_BACKGROUND])
SECTION_HEADERS_FOR_REPORT.update([x.lower() for x in SECTION_HEADERS_FOR_REPORT])
IGNORABLE_HEADERS.update([x.lower() for x in IGNORABLE_HEADERS])

KNOWN_HEADERS = SECTION_HEADERS_FOR_PATIENT_BACKGROUND |\
                SECTION_HEADERS_FOR_REPORT_BACKUP |\
                SECTION_HEADERS_FOR_REPORT |\
                IGNORABLE_HEADERS
KNOWN_HEADERS.update(x.lower() for x in [
    'EXAMINATION:',
    'COMPARISON:',
    'COMPARISONS:',
    'COMPARISON STUDY:',
    'Comparisons:',
    'COMPARISON:',
    'COMPARISONS:',
    'Comparison:',
    'COMPARISION:',
    'COMPARISON STUDY:',
    'COMPARISON TO PRIOR STUDY:',
    'COMPARISON STUDIES:',
    'COMPARISON EXAMS:',
    'COMPARRISON:',
    'COMPARISON comparison:',
    'COMPARISON CHEST:',
    'Comp:',
    'The comparison:',
    'PRIOR EXAM:',
    'CT chest:',
    '(HEADERLESS PARAGRAPH)',
    'Correction:',
    'The:',
    'Support Devices:',
    'Lungs:', 
    'Pleura:', 
    'Heart:',
    'Lines and Tubes:',
    'Lines and tubes:',
    'Mediastinum:',
    'Mediastinum and hila:',
    'Osseous structures:', 
    'Other findings:',
    'Bony thorax:',
    'RIGHT RIBS:',
    'Multitrauma:',
    'Pelvis:',
    'Postoperative changes including:',
    'Abdomen:',
    'Tip of the new intra-aortic:',
    'Multiple pulmonary abnormalities include:',
    'The following support devices are identified:',
    'CHEST:',
    'ADDENDUM Clarification:',
    'ADDENDUM IMPRESSION:',
    'AMENDMENT:',
    'ADDENDUM:',
    'ADDENDUM Correction:',
    'ADDENDUM To clarify the above read:',
    'ADDENDUM The correct report should read as follows:',
    'ADDENDUM Please read of the IMPRESSION as following:',
    'The correct IMPRESSION heading should be inserted after:',
    'Allowing for these limitations:',
    'The impression should read:',
    'Impression should read:',
     'Right lower lung opacity has increased:',
    'Final:',
    'CTA chest:',
    'Note:',
    'Comment:',
    'NOTIFICATIONS:',
    'NOTIFICATION:',
    'MAIN REPORT:',
    'TECHNIQUE:',
    'PROCEDURE:',
    'NOTE:',
    'COMMENTS:',
    'COMMENT:',
    'PORTABLE CHEST:',
    'REFERENCE EXAM:',
    'REFERENCE EXAMINATION:',
    'COMPARISON EXAM:',
    'COMPARISON FILM:',
    'COMPARISON FILMS:',
    'CHEST COMPARISON FILM:',
])

In [2031]:
for i in tqdm(range(i+1, len(report_paths))):
    rp = report_paths[i]
    try:
        sections = split_report_into_sections(rp)
        for k in sections.keys():
            if not k.startswith('(AFTER)'):
                assert k.lower() in KNOWN_HEADERS, k
        assert len(sections) > 0
        
        tmp = extract_report_and_patient_background(rp)
        assert len(tmp['report']) > 0
#         assert len(tmp['background']) > 0
    except AssertionError:
        print(i, rp)
        raise

  3%|▎         | 3609/120600 [00:12<06:48, 286.35it/s]

110844 /mnt/data/mimic-cxr/files/p14/p14065514/s56763333.txt


AssertionError: 

In [1941]:
re_invalid.search('''AP single view of the chest has been obtained with patient in semi-upright
 position.  The heart size is within normal limits paying attention to AP
 technique.  No typical configurational abnormality is seen.  Thoracic aorta
 unremarkable with the exception of some calcium deposits in the wall at the
 level of the arch.  The pulmonary vasculature is not congested.  No signs of
 acute parenchymal infiltrates are present.  The right lateral pleural sinus is
 free; however, there is a small amount of pleural density blunting the left
 lateral pleural sinus, and some crowded vasculature on the left base suggests
 some degree of atelectasis.  Acute new infiltrates cannot be identified on
 this portable single view chest examination.  No pneumothorax exists in the
 apical area.  Our records do not include a previous chest examination
 available for comparison.''')

<_sre.SRE_Match object; span=(789, 796), match='records'>

In [2033]:
import random

In [2054]:
rp = random.choice(report_paths)
try:
    sections = split_report_into_sections(rp)
    for k in sections.keys():
        if not k.startswith('(AFTER)'):
            assert k.lower() in KNOWN_HEADERS, k
    assert len(sections) > 0

    tmp = extract_report_and_patient_background(rp)
    assert len(tmp['report']) > 0
    assert len(tmp['background']) > 0
except AssertionError:
    print(i, rp)
    raise

In [2055]:
pprint(extract_report_and_patient_background(rp, True))

                                 FINAL REPORT
 CHEST RADIOGRAPH
 
 INDICATION:  Recent pneumonia, persistent cough, evaluation.
 
 COMPARISON:  No comparison available at the time of dictation.
 
 FINDINGS:  Normal lung volumes.  Normal size of the cardiac silhouette. 
 Normal appearance of the lung parenchyma.  No evidence of pneumonia.  Small
 right pericardial fat pad.  No pleural effusions.

OrderedDict([('(HEADERLESS PARAGRAPH)',
              ['                                 FINAL REPORT\n'
               ' CHEST RADIOGRAPH',
               '\n \n ',
               ' \n ']),
             ('INDICATION:',
              ['  Recent pneumonia, persistent cough, evaluation.']),
             ('(AFTER) INDICATION:', ['\n \n ', ' \n ']),
             ('COMPARISON:',
              ['  No comparison available at the time of dictation.']),
             ('(AFTER) COMPARISON:', ['\n \n ', ' \n ']),
             ('FINDINGS:',
              ['  Normal lung volumes.  Normal size of the cardiac 

In [1964]:
re_invalid = load_invalid_patterns_regex()

In [1829]:
re_invalid.search('The impression and recommendation above was entered by Dr. on at '
             '11:12 into the Department of Radiology critical communications '
             'system for direct communication to the referring provider.')

<_sre.SRE_Match object; span=(52, 57), match='by Dr'>

In [1500]:
from medvqa.metrics.medical.med_completeness import MEDICAL_TERMS_PATH
from medvqa.utils.files import read_lines_from_txt
from nltk import wordpunct_tokenize

medical_terms = set(read_lines_from_txt(MEDICAL_TERMS_PATH))

In [1501]:
def contains_medical_terms(text, k):
    count = 0
    for x in wordpunct_tokenize(text.lower()):
        if x in medical_terms:
            count += 1
            if count >= k: return True
    return False

In [1817]:
import re
from medvqa.datasets.qa_pairs_extractor import REGULAR_EXPRESSIONS_FOLDER

def load_invalid_patterns_regex():
    pattern = ''
    with open(os.path.join(REGULAR_EXPRESSIONS_FOLDER, 'invalid_sentence_patterns.txt')) as f:
        for line in f.readlines():
            if len(pattern) > 0:
                pattern += '|'
            pattern += f'({line.strip()})'
    return re.compile(pattern, re.IGNORECASE)

In [1844]:
def contains_no_invalid_patterns(text):
    return not re_invalid.search(text)

In [1521]:
contains_medical_terms('in comparison to previous chest x-ray support devices radiography', 1)

True

In [1607]:
'3dAsdfasdf'[0].isalpha()

False

In [1955]:
def extract_report_and_patient_background(report_path, debug=False):
    sections = split_report_into_sections(report_path, debug=debug)
    if debug:
        pprint(sections)
    report_chunks = []
    background_chunks = []
    
    if debug:
        print('===================================================')
    
    for k_, v in sections.items():
        k = k_.lower()
        
        if k in SECTION_HEADERS_FOR_PATIENT_BACKGROUND:
            
            if type(v) is list:
                for i in range(len(v)):
                    v[i] = v[i].replace('_', ' ')
                    v[i] = v[i].strip()
                    if len(v[i]) > 0 and v[i][-1] != '.': v[i] += '.'
                if k == '(after) wet read:':
                    v = ' '.join(x for x in v if len(x) > 0 and contains_medical_terms(x, 1))
                else:
                    v = ' '.join(x for x in v if len(x) > 0)
            else:
                v = v.replace('_', ' ').strip()
                if k == '(after) wet read:' and not contains_medical_terms(v, 1):
                    v = ''                    
            v = ' '.join(v.split())
            if len(v) == 0 or v == '.': continue
            if v[-1] != '.': v += '.'
            background_chunks.append((k, v))
            
        elif k in SECTION_HEADERS_FOR_REPORT:
            
            if type(v) is list:
                for i in range(len(v)):
                    v[i] = v[i].replace('_', ' ')
                    v[i] = v[i].strip()
                    if len(v[i]) > 0 and v[i][-1] != '.': v[i] += '.'
                v = ' '.join(x for x in v if len(x) > 0)
            else:
                v = v.replace('_', ' ').strip()
            v = ' '.join(v.split())
            if len(v) == 0 or v == '.': continue
            if v[-1] != '.': v += '.'            
            
            report_chunks.append((k, v))
            
        elif k not in IGNORABLE_HEADERS:
            
            if type(v) is list:
                for i in range(len(v)):
                    v[i] = v[i].replace('_', ' ')
                    v[i] = v[i].strip()
#                     if len(v[i]) > 0 and v[i][-1] != '.' and (i+1 == len(v) or len(v[i+1]) == 0 or
#                                                               v[i+1][0].isalpha()): v[i] += '.'
                v = ' '.join(x for x in v if len(x) > 0 and\
                             contains_medical_terms(x, 2) and\
                             contains_no_invalid_patterns(x))
            elif contains_medical_terms(v, 2) and contains_no_invalid_patterns(v):
                v = v.replace('_', ' ').strip()
            else:
                v = ''
            v = ' '.join(v.split())
            if len(v) == 0 or v == '.': continue
            if v[-1] != '.': v += '.'
                
            if (not k.startswith('(after)') and contains_medical_terms(k, 1)):
                v = f'{k_} {v}'
            
            report_chunks.append((k, v))
            
    if len(report_chunks) == 0:
        for i, p in enumerate(background_chunks):
            if contains_medical_terms(p[1], 3) and contains_no_invalid_patterns(p[1]):
                report_chunks.append(p)
                background_chunks[i] = None
        background_chunks = [p for p in background_chunks if p is not None]

    return dict(
        report = report_chunks,
        background = background_chunks,
    )

In [756]:
len("""Chest tubes and right pleural catheter are in unchanged position.  There is no
 evident pneumothorax.  Bibasilar opacities have increased.  This is partially""")

157

In [23]:
len(split_report_into_sections(report_paths[0]))

6

In [23]:
from medvqa.datasets.mimiccxr import load_mimiccxr_reports_detailed_metadata
from medvqa.utils.files import list_filepaths_with_prefix_and_timestamps, read_txt
from medvqa.utils.logging import print_bold

In [16]:
list_filepaths_with_prefix_and_timestamps('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background')

[('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json__detailed_metadata.pkl',
  '2023-08-29 10:04:02'),
 ('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json',
  '2023-06-12 17:41:46'),
 ('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230610_132343.json',
  '2023-06-10 13:23:46')]

In [1]:
!ls -lh "/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json__detailed_metadata.pkl"

-rw-rw-r-- 1 pamessina pamessina 147M Aug 29  2023 /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json__detailed_metadata.pkl


In [17]:
metadata = load_mimiccxr_reports_detailed_metadata('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json')

In [18]:
i = metadata['study_ids'].index(53531026)
i

5599

In [25]:
print(metadata['filepaths'][i])
print()
print(read_txt(metadata['filepaths'][i]))
print()
print_bold('Background:')
print()
print(metadata['backgrounds'][i])
print()
print_bold('Findings:')
print()
print(metadata['findings'][i])
print()
print_bold('Impression:')
print()
print(metadata['impressions'][i])

/mnt/data/mimic-cxr/files/p10/p10382575/s53531026.txt

                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)
 
 INDICATION:  ___ year old woman with new cirrhosis with decompensation  // ?
 Pneumonia      ? Pneumonia
 
 COMPARISON:  No comparison studies.  Please note that comparison to old with
 images can be helpful to detect subtle interval change.
 
 FINDINGS: 
 
 PA and lateral views of the chest ___ at 13:19 are submitted.
 
 IMPRESSION: 
 
 There is minimal blunting of left costophrenic angle and patchy opacity at the
 left base which may reflect a combination of atelectasis and or scarring in
 the setting of a small effusion or chronic pleural thickening.  An early
 infectious process should also be considered.  Clinical correlation is
 recommended.  No pulmonary edema or pneumothorax.  Overall cardiac and
 mediastinal contours are within normal limits.


Background:

indication: year old woman with new cirrhosis with decompensation // ? Pneumonia ? Pn

In [29]:
extract_background_findings_and_impression(metadata['filepaths'][i], debug=True)

                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)
 
 INDICATION:  ___ year old woman with new cirrhosis with decompensation  // ?
 Pneumonia      ? Pneumonia
 
 COMPARISON:  No comparison studies.  Please note that comparison to old with
 images can be helpful to detect subtle interval change.
 
 FINDINGS: 
 
 PA and lateral views of the chest ___ at 13:19 are submitted.
 
 IMPRESSION: 
 
 There is minimal blunting of left costophrenic angle and patchy opacity at the
 left base which may reflect a combination of atelectasis and or scarring in
 the setting of a small effusion or chronic pleural thickening.  An early
 infectious process should also be considered.  Clinical correlation is
 recommended.  No pulmonary edema or pneumothorax.  Overall cardiac and
 mediastinal contours are within normal limits.

OrderedDict([('(HEADERLESS PARAGRAPH)',
              ['                                 FINAL REPORT']),
             ('EXAMINATION:', ['  CHEST (PA AND 

{'background': 'indication: year old woman with new cirrhosis with decompensation // ? Pneumonia ? Pneumonia.',
 'findings': 'PA and lateral views of the chest at 13:19 are submitted. There is minimal blunting of left costophrenic angle and patchy opacity at the left base which may reflect a combination of atelectasis and or scarring in the setting of a small effusion or chronic pleural thickening. An early infectious process should also be considered. Clinical correlation is recommended. No pulmonary edema or pneumothorax. Overall cardiac and mediastinal contours are within normal limits.',
 'impression': ''}

In [12]:
import pandas as pd
from medvqa.datasets.mimiccxr import MIMICCXR_METADATA_CSV_PATH, get_mimiccxr_report_path, get_imageId2partId
from medvqa.utils.files_utils import read_txt

In [10]:
imageId2partId = get_imageId2partId()

In [3]:
df = pd.read_csv(MIMICCXR_METADATA_CSV_PATH)

In [6]:
df[df.study_id == 55343330]

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
1513,8bb728f7-59213a2c-1dab5f9e-875589a2-a308d030,10035780,55343330,Performed Desc,LL,2021,1560,21310807,153542.0,CHEST (PA AND LAT),left lateral,Recumbent
1514,efe8834c-2f3e9afb-011c86bc-4506aff3-204a3256,10035780,55343330,Performed Desc,PA,1633,1813,21310807,153542.0,CHEST (PA AND LAT),postero-anterior,Recumbent


In [7]:
df[df.subject_id == 10035780]

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
1509,5b434f5b-7a561c59-c4869996-f2707d0f-2eff09f6,10035780,52502485,CHEST (PORTABLE AP),AP,3056,2544,21310725,174711.000,CHEST (PORTABLE AP),antero-posterior,Erect
1510,b872a0c8-1a7c1ded-43ff067d-47d52e47-3fa5e874,10035780,53087369,CHEST (PA AND LAT),AP,2843,2480,21320609,172416.156,CHEST (PA AND LAT),antero-posterior,Erect
1511,e83fe62e-4d4818fa-139761b6-c1d1d171-ed224714,10035780,53087369,CHEST (PA AND LAT),LATERAL,3056,2544,21320609,172416.156,CHEST (PA AND LAT),lateral,Erect
1512,1ca83fb4-cebdf1c6-0ebd8f9d-7cb0ff82-09395c91,10035780,53120593,CHEST (PORTABLE AP),AP,2544,3056,21320514,154031.984,CHEST (PORTABLE AP),antero-posterior,Erect
1513,8bb728f7-59213a2c-1dab5f9e-875589a2-a308d030,10035780,55343330,Performed Desc,LL,2021,1560,21310807,153542.000,CHEST (PA AND LAT),left lateral,Recumbent
1514,efe8834c-2f3e9afb-011c86bc-4506aff3-204a3256,10035780,55343330,Performed Desc,PA,1633,1813,21310807,153542.000,CHEST (PA AND LAT),postero-anterior,Recumbent
1515,06bb8f91-598d12db-baa0c122-aa071d90-d976c5a7,10035780,59076224,NaN,LATERAL,3056,2544,21311109,91234.203,lateral,Erect,NaN
1516,81601813-88063656-fdacf6b6-2f97c7dd-ace5238e,10035780,59076224,NaN,AP,3056,2544,21311109,91234.203,antero-posterior,Erect,NaN


In [15]:
dicom_id = '81601813-88063656-fdacf6b6-2f97c7dd-ace5238e'
part_id = imageId2partId[dicom_id]
subject_id = 10035780
study_id = 52502485
print(read_txt(get_mimiccxr_report_path(part_id, subject_id, study_id)))

 WET READ: ___ ___ 8:14 AM
  
  
  
  No acute cardiopulmonary process.
 WET READ VERSION #1 ___ ___ 1:21 AM
  No acute cardiopulmonary process.
 ______________________________________________________________________________
                                 FINAL REPORT
 EXAMINATION:  CHEST (PORTABLE AP)
 
 INDICATION:  ___ year old woman with CKD admitted for HD initiation  // Require
 (-) ppd and (-) CXR for outpatient HD.
 
 COMPARISON:  ___.
 
 IMPRESSION: 
 
 Low lung volumes.  Borderline size of the cardiac silhouette.  Mild elongation
 of the descending aorta.  No pneumonia, no pneumothorax, no pleural effusions.
 No evidence of TB.



In [16]:
dicom_id = 'efe8834c-2f3e9afb-011c86bc-4506aff3-204a3256'
part_id = imageId2partId[dicom_id]
subject_id = 10035780
study_id = 55343330
print(read_txt(get_mimiccxr_report_path(part_id, subject_id, study_id)))

                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)
 
 INDICATION:  ___ year old woman with new leukocytosis.  // R/O infection
 
 TECHNIQUE:  Chest PA and lateral
 
 COMPARISON:  Chest radiograph dated ___.
 
 FINDINGS: 
 
 There is a dialysis catheter overlying the right chest with the tip in the
 cavoatrial junction.  Heart size is stable.  The mediastinal and hilar
 contours are stable. The pulmonary vasculature is normal. Lungs are clear.  No
 pleural effusion or pneumothorax is seen. There are no acute osseous
 abnormalities.
 
 IMPRESSION: 
 
 No evidence of pneumonia.

